In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import torch.distributed as dist

In [2]:
SEED=1
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [3]:
torch.cuda.device_count()

1

In [4]:
START_EPOCH = 0
ARCH = 'resnet18'
EPOCHS = 200
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=256
VAL_BATCH=256
WORKERS=8
TRAINDIR="/workspace/storage/train"
VALDIR="/workspace/storage/val"

In [5]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [6]:
URL = 'tcp://3.19.70.152:1234'
BACKEND = 'nccl'
WORLD_SIZE = 1
RANK = 0

dist.init_process_group(backend = BACKEND, init_method= URL,
                                 world_size= WORLD_SIZE, rank=RANK)

In [7]:
GPU = 0
torch.cuda.set_device(GPU)
cudnn.benchmark = True

In [8]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [9]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

In [10]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [11]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [12]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [13]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    #lr = LR * (0.1 ** (epoch // 30))
    lr = LR * (0.1 ** (epoch // 17))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [14]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [15]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [16]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [17]:
IMG_SIZE = 224
NUM_CLASSES = 1000
#model = models.resnet18()
model = models.__dict__[ARCH]()
inf = model.fc.in_features
model.fc = nn.Linear(inf, NUM_CLASSES)
model.cuda(GPU)
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

In [18]:
criterion = nn.CrossEntropyLoss().cuda(GPU)
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
])

In [19]:
# train_dataset = datasets.ImageFolder(
#     TRAINDIR, transform=transform_train)

train_dataset = datasets.ImageFolder(
    TRAINDIR,
    transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])
)

In [20]:
# transform_val = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
# ])

val_dataset = datasets.ImageFolder(
    VALDIR,
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])
)

In [21]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [22]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [23]:
%pip install -q wandb
import wandb
wandb.login()

Note: you may need to restart the kernel to use updated packages.


wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [24]:
wandb.init(project="john-andrus-HW9-1node")

wandb: Currently logged in as: johnmandrus (use `wandb login --relogin` to force relogin)


In [ ]:
best_acc1 = 0

for epoch in range(START_EPOCH, EPOCHS):
    
    t1 = time.time()
    
    #Implement Learning Rate Adjustment
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    
    wandb.log({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    })
    
    t2 = time.time()
    
    print("Epoch Time:",(t2-t1)/60,"minutes")

Epoch: [0][   0/5005]	Time 10.354 (10.354)	Data  3.641 ( 3.641)	Loss 6.9477e+00 (6.9477e+00)	Acc@1   0.00 (  0.00)	Acc@5   1.95 (  1.95)
Epoch: [0][  50/5005]	Time  0.773 ( 0.943)	Data  0.000 ( 0.072)	Loss 6.8990e+00 (7.0538e+00)	Acc@1   0.39 (  0.22)	Acc@5   1.17 (  0.86)
Epoch: [0][ 100/5005]	Time  0.826 ( 0.874)	Data  0.000 ( 0.042)	Loss 6.7530e+00 (6.9402e+00)	Acc@1   1.56 (  0.33)	Acc@5   3.91 (  1.27)
Epoch: [0][ 150/5005]	Time  1.336 ( 0.968)	Data  0.000 ( 0.032)	Loss 6.6977e+00 (6.8640e+00)	Acc@1   0.00 (  0.44)	Acc@5   2.73 (  1.65)
Epoch: [0][ 200/5005]	Time  1.329 ( 1.059)	Data  0.000 ( 0.028)	Loss 6.6168e+00 (6.8077e+00)	Acc@1   0.78 (  0.48)	Acc@5   3.52 (  1.88)
Epoch: [0][ 250/5005]	Time  1.332 ( 1.124)	Data  0.000 ( 0.026)	Loss 6.5420e+00 (6.7567e+00)	Acc@1   1.56 (  0.55)	Acc@5   4.30 (  2.14)
Epoch: [0][ 300/5005]	Time  1.344 ( 1.163)	Data  0.000 ( 0.025)	Loss 6.4127e+00 (6.7094e+00)	Acc@1   1.17 (  0.62)	Acc@5   4.69 (  2.45)
Epoch: [0][ 350/5005]	Time  1.285 ( 1.196

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: [0][ 550/5005]	Time  1.277 ( 1.267)	Data  0.000 ( 0.022)	Loss 6.0636e+00 (6.5066e+00)	Acc@1   1.17 (  1.03)	Acc@5   5.08 (  3.88)
Epoch: [0][ 600/5005]	Time  1.403 ( 1.279)	Data  0.000 ( 0.022)	Loss 6.0621e+00 (6.4690e+00)	Acc@1   2.34 (  1.14)	Acc@5   7.42 (  4.22)
Epoch: [0][ 650/5005]	Time  1.347 ( 1.288)	Data  0.000 ( 0.021)	Loss 6.0173e+00 (6.4347e+00)	Acc@1   1.95 (  1.24)	Acc@5   8.59 (  4.54)
Epoch: [0][ 700/5005]	Time  1.275 ( 1.295)	Data  0.000 ( 0.021)	Loss 6.0002e+00 (6.4007e+00)	Acc@1   4.69 (  1.34)	Acc@5  10.55 (  4.85)
Epoch: [0][ 750/5005]	Time  2.294 ( 1.304)	Data  0.000 ( 0.021)	Loss 5.9593e+00 (6.3690e+00)	Acc@1   2.73 (  1.46)	Acc@5  10.94 (  5.17)
Epoch: [0][ 800/5005]	Time  1.449 ( 1.309)	Data  0.000 ( 0.021)	Loss 5.8184e+00 (6.3371e+00)	Acc@1   4.69 (  1.56)	Acc@5  12.50 (  5.48)
Epoch: [0][ 850/5005]	Time  1.335 ( 1.315)	Data  0.000 ( 0.021)	Loss 5.7869e+00 (6.3070e+00)	Acc@1   2.73 (  1.67)	Acc@5   9.77 (  5.79)
Epoch: [0][ 900/5005]	Time  1.334 ( 1.318